This Kernel for someone want to deep dive into image classification. I use CNN for classification model. If you found this Kernel helpful please up vote it. If you have some feedback and question don't forget to comment below. 

I have simplier model with 
* https://www.kaggle.com/uysimty/get-start-image-classification

# Import Library

In [1]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator # 이미지 데이터의 배치를 생성, 이미지 전처리에 사용
from keras.utils import to_categorical # 여러 개의 Y값을 0과 1로만 이루어진 형태로 바꿔주는 one-hot-encoding
from sklearn.model_selection import train_test_split # scikit-learn 패키지 중 model_selection에 데이터 분할을 위한 train_test_split 함수가 있음.
import matplotlib.pyplot as plt
import random
import os # os.listdir을 쓰기 위해서


2023-01-07 06:35:40.317461: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 06:35:40.437525: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import urllib

import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

import tensorflow as tf

# Define Constants

In [3]:
opener = urllib.request.build_opener()
opener.addheaders = [("User-agent", "Mozilla/5.0")]
urllib.request.install_opener(opener)

In [4]:
FAST_RUN = False # 아래에 사용됨
IMAGE_WIDTH=128 # 이미지 넓이(행)
IMAGE_HEIGHT=128 # 이미지 높이(열)
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3 # 이미지 channel(depth)

In [5]:
BATCHSIZE = 256
CLASSES = 2
EPOCHS = 7
N_TRAIN_EXAMPLES = 3000
STEPS_PER_EPOCH = int(N_TRAIN_EXAMPLES / BATCHSIZE)
VALIDATION_STEPS = 30

In [6]:
global train_df
global validate_df

# Prepare Traning Data

In [7]:
filenames = os.listdir("data/train") # os.listdir은 디렉토리 내에 있는 모든 파일 및 디렉토리 리스트를 가져와서 indexing 한다.
categories = [] # 빈 list 한 개 만든다.
for filename in filenames:              # filenames에 있는 list들을 한 개씩 받아온다.                       
    category = filename.split('.')[0]   # . 을 기준으로 나누고, '0'위치에 있는 것을 category 변수에 저장
    if category == 'dog':
        categories.append(1) # 강아지이면 1을 넣고 (append : 덧붙이다)
    else:
        categories.append(0) # 강아지가 아니면(고양이면) 0을 넣어라

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
}) # 데이터프레임 만들기 -  filename이라는 col에 filenames value 넣기, category라는 col에 categories value 넣기

# Prepare data

Because we will use image genaretor `with class_mode="categorical"`. We need to convert column category into string. Then imagenerator will convert it one-hot encoding which is good for our classification. 

So we will convert 1 to dog and 0 to cat

In [8]:
df["category"] = df["category"].replace({0: 'cat', 1: 'dog'})

# replace를 이용해서 0은 cat으로 1은 dog로 바꿔라.
# 이유: image generator를 사용할 것이기 때문이고,
#       image generator는 classification에 적합한 one-hot encoding을 변환한다.

In [9]:
df.head() # category 확인

,filename,category
0,cat.5446.jpg,cat
1,cat.7408.jpg,cat
2,dog.24.jpg,dog
3,cat.10010.jpg,cat
4,cat.12418.jpg,cat


In [10]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
# df: 분할시킬 data (DataFrame)
# test_size: test dataset의 비율이나 갯수(default=0.25) -> 여기서는 validation set이라 할 수 있다.
# random state: 데이터 분할시 셔플이 이루어지는데 이를 위한 시드값
# shuffle: 셔플여부설정 (default=True)

train_df = train_df.reset_index(drop=True) # index를 reset함 -> drop: index로 세팅한 열을 데이터프레임내에서 삭제할지 여부를 결정한다.
validate_df = validate_df.reset_index(drop=True)

In [11]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15

In [12]:
# 제가 뭔지 확인해 본거에요.
print(train_df.shape) 
print(validate_df.shape) 

(20000, 2)
(5000, 2)


# Traning Generator

In [13]:
def train_dataset(train_df):
    # 신경망 모델의 성능을 높이기 위한 위한 "데이터 부풀리기(Data augmentation)"
    train_datagen = ImageDataGenerator(
        rotation_range=15,
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1
    )
    # rotation_range : 지정된 각도 "범위"내에서 임의로 원본이미지를 회전시킨다. // 단위:(도) -> 여기서는 15니까, 0도에서 15도 사이
    # rescale : byte와 관련이 있는듯 255라는 숫자가
    # shear_range : 밀림 강도 범위내에서 임의로 원본이미지를 변형시킴

    # zoom_range : 지정된 확대/축소 범위내에서 임의로 원본이미지를 확대/축소. “1 - 수치”부터 “1 + 수치”사이 범위로 확대/축소.
    #              예를 들어 0.2이라면, 0.8배에서 1.2배 크기 변화를 시킵니다.

    # horizontal_flip : 수평방향으로 뒤집느냐 마느냐

    # width_shift_range : 지정된 "수평방향" 이동 범위내에서 임의로 원본이미지를 이동시킴. 수치는 "전체 넓이"의 비율(실수)로 나타냄.
    #                     예를 들어 0.1이고 전체 넓이가 100이면, 10픽셀 내외로 "좌우" 이동.

    # height_shift_range : 지정된 "수직방향" 이동 범위내에서 임의로 원본이미지를 이동시킴. 수치는 "전체 높이"의 비율(실수)로 나타냄..
    #                       예를 들어 0.1이고 전체 높이가 100이면, 10픽셀 내외로 "상하" 이동.


    # 'dataframe'과 '디렉토리'의 위치를 전달받아 증강/정규화된 데이터의 "배치"를 생성.
    train_generator = train_datagen.flow_from_dataframe( 
        train_df, # DataFrame (전체 중에 80%만 있음)
        "data/train/", # 문자열, 이미지를 읽을 '디렉토리'의 경로 (전체 data가 있음) 
        x_col='filename', # train_df 데이터프레임에 filename col의 값들을 가져옴
        y_col='category', # train_df 데이터프레임에 category col의 값들을 가져옴
        target_size=IMAGE_SIZE, # 
        class_mode='categorical', # 2D numpy array of one-hot encoded labels. Supports multi-label output.
        batch_size=batch_size # batch size
    )
    return train_generator
    

In [14]:
train_ds = train_dataset(train_df)
train_ds

Found 20000 validated image filenames belonging to 2 classes.


### Validation Generator

In [15]:
def validate_dataset(validate_df):
    # 위에 train_datagen과는 다르게 rescale만 해줌. (검증하는 거니까!)
    # rescale 하는 이유: 정규화 과정임. image가 0~255까지 값을 가지는 2차원 배열인데, 0~255 사이의 값을
    #                   0.0과 1.0사이의 값으로 바꾸기 위함이다.
    #                   활성화함수 및 오류역전파 알고리즘은 0.0과 1.0사이의 값을 좋아하기 때문이다.
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # 위에 있는 train_generator와 같다
    validation_generator = validation_datagen.flow_from_dataframe(
        validate_df, 
        "data/train/", 
        x_col='filename',
        y_col='category',
        target_size=IMAGE_SIZE,
        class_mode='categorical',
        batch_size=batch_size
    )

    return validation_generator 

In [16]:
validate_ds = validate_dataset(validate_df)
validate_ds

Found 5000 validated image filenames belonging to 2 classes.


Seem to be nice 

* **Input Layer**: It represent input image data. It will reshape image into single diminsion array. Example your image is 64x64 = 4096, it will convert to (4096,1) array.
* **Conv Layer**: This layer will extract features from image.
* **Pooling Layer**: This layerreduce the spatial volume of input image after convolution.
* **Fully Connected Layer**: It connect the network from a layer to another layer
* **Output Layer**: It is the predicted values layer. 

In [17]:
from keras.applications import VGG16
def create_model(trial):
    # Hyperparameters to be tuned by Optuna.
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    momentum = trial.suggest_float("momentum", 0.0, 1.0)
    units = trial.suggest_categorical("units", [32, 64, 128, 256, 512])
    
    conv_base = VGG16(weights='imagenet',
                    include_top=False,
                    input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))

    model = tf.keras.Sequential() # 가장 자주 사용하는 구조인 층을 순서대로 쌓아 올린 네트워크{keras 사용 - 모델을 정의(생성)}
    conv_base.trainable = False  # Freeze weights of conv_base layers (VGG16)

    # Convolution
    # 인공신경망 모델을 효율적으로 학습시키기 윈한 개선 방법들 (BatchNormalization, Dropout, ModelEnsemble)
    model.add(conv_base)
    model.add(tf.keras.layers.Flatten()) # - 이 층에는 학습되는 가중치가 없고 데이터를 변환하기만 한다.(1차원 vector으로 바꿈)
                        # - 이 층은 하나의 layer에 있는 모든 neuron을 또 다른 layer의 모든 neuron과 연결 시켜준다.
    model.add(tf.keras.layers.Dense(256, activation='relu')) # dense로 층을 쌓음 - 첫번째 Dense층은 512개의 노드를 가짐
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(2, activation='softmax')) # 2: because we have cat and dog classes
                                            # 마지막 층은 2개의 노드의 소프트맥스층 : 이 층은 2개의 확률을 반환하고 반환된 값의 전체 합은 1이다. 
                                            # 각 노드는 현재 이미지가 2개 클래스 중 하나에 속할 확률을 출력


    # # Compose neural network with one hidden layer.
    # model = tf.keras.Sequential()
    # model.add(tf.keras.layers.Flatten())
    # model.add(tf.keras.layers.Dense(units=units, activation=tf.nn.relu))
    # model.add(tf.keras.layers.Dense(CLASSES, activation=tf.nn.softmax))

    # Compile model.
    model.compile(
        optimizer=tf.keras.optimizers.SGD(
            learning_rate=learning_rate, momentum=momentum, nesterov=True
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [18]:
def objective(trial):
    global train_df
    global validate_df
    # Clear clutter from previous TensorFlow graphs.
    tf.keras.backend.clear_session()

    # Metrics to be monitored by Optuna.
    if tf.__version__ >= "2":
        monitor = "val_accuracy"
    else:
        monitor = "val_acc"

    # Create tf.keras model instance.
    model = create_model(trial)

    # Create dataset instance.
    ds_train = train_dataset(train_df)
    ds_eval = validate_dataset(validate_df)

    # Create callbacks for early stopping and pruning.
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=3),
        TFKerasPruningCallback(trial, monitor),
    ]

    # Train model.
    history = model.fit(
        ds_train,
        epochs=EPOCHS,
        steps_per_epoch=STEPS_PER_EPOCH,
        validation_data=ds_eval,
        validation_steps=VALIDATION_STEPS,
        callbacks=callbacks,
    )

    return history.history[monitor][-1]

In [19]:
def show_result(study):

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [20]:
def main():

    study = optuna.create_study(
        direction="maximize", pruner=optuna.pruners.MedianPruner(n_startup_trials=2)
    )

    study.optimize(objective, n_trials=25, timeout=600)

    show_result(study)

In [21]:
if __name__ == "__main__":
    main()

[I 2023-01-07 06:35:46,750] A new study created in memory with name: no-name-b35c8304-bac5-46e4-bdc1-1f5242be6c80
2023-01-07 06:35:46.851512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 06:35:47.445427: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22284 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Epoch 1/7


2023-01-07 06:35:50.164972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-01-07 06:35:52.142144: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-01-07 06:35:52.145813: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fb9a0014520 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-01-07 06:35:52.145866: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-01-07 06:35:52.229340: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 [==============================] - 6s 162ms/step - loss: 0.8147 - accuracy: 0.6667 - val_loss: 0.5479 - val_accuracy: 0.7156
Epoch 2/7
11/11 [==============================] - 2s 159ms/step - loss: 0.6672 - accuracy: 0.7576 - val_loss: 0.4516 - val_accuracy: 0.7667
Epoch 3/7
11/11 [==============================] - 2s 158ms/step - loss: 0.6736 - accuracy: 0.7394 - val_loss: 0.3787 - val_accuracy: 0.8467
Epoch 4/7
11/11 [==============================] - 2s 148ms/step - loss: 0.6333 - accuracy: 0.7333 - val_loss: 0.7870 - val_accuracy: 0.6422
Epoch 5/7
11/11 [==============================] - 2s 157ms/step - loss: 0.5996 - accuracy: 0.7273 - val_loss: 1.0292 - val_accuracy: 0.5733
Epoch 6/7
11/11 [==============================] - 2s 153ms/step - loss: 0.6116 - accuracy: 0.7333 - val_loss: 0.8610 - val_accuracy: 0.6422


[I 2023-01-07 06:36:02,556] Trial 0 finished with value: 0.6422222256660461 and parameters: {'learning_rate': 0.010146995548684241, 'momentum': 0.17612938714458037, 'units': 128}. Best is trial 0 with value: 0.6422222256660461.


...

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Epoch 1/7
11/11 [==============================] - ETA: 0s - loss: 0.7709 - accuracy: 0.6303

[I 2023-01-07 06:37:50,931] Trial 23 pruned. Trial was pruned at epoch 0.


Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Epoch 1/7
11/11 [==============================] - ETA: 0s - loss: 0.8125 - accuracy: 0.6424

[I 2023-01-07 06:37:54,352] Trial 24 pruned. Trial was pruned at epoch 0.


Study statistics: 
  Number of finished trials:  25
  Number of pruned trials:  21
  Number of complete trials:  4
Best trial:
  Value:  0.8422222137451172
  Params: 
    learning_rate: 0.002405219201927165
    momentum: 0.0509660878469152
    units: 512


# Save Model

In [100]:
# 한번 training을 할 때 대용량 데이터를 트레이닝 한 뒤 model을 save하지 않으면 다시 처음부터
# 트레이닝을 진행해야한다.
model.save_weights("vgg.h5") # weight를 h5 파일 포맷으로 만들어 저장하기
                               # keras에서는 모델과 weights의 재사용을 위해 이를 파일형태로 저장하는 라이브러리를 제공하며,
                               # 이를 통해 모델과 weights를 파일 형태로 저장하고 불러올 수가 있다.

In [102]:
test_filenames = os.listdir("data/test") # test에 있는 list들을 가져옴
test_df = pd.DataFrame({                                       # 데이터프레임을 생성
    'filename': test_filenames                                 # filename이라는 col에 test_filenames값을 가져옴
})
nb_samples = test_df.shape[0] # test dataset data 갯수

In [103]:
test_df.shape

(12500, 1)

# Create Testing Generator

In [104]:
# rescale만 해줌. (검증하는 거니까!)
# rescale 하는 이유: 정규화 과정임. image가 0~255까지 값을 가지는 2차원 배열인데, 0~255 사이의 값을
#                   0.0과 1.0사이의 값으로 바꾸기 위함이다.
#                   활성화함수 및 오류역전파 알고리즘은 0.0과 1.0사이의 값을 좋아하기 때문이다.
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "data/test/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

# shuffle을 쓰지 않음

Found 12500 validated image filenames.


# Predict

In [105]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))
# test_generator 자리 : 입력 샘플의 batch를 생성하는 생성기
# steps : 중지되기 전까지 generator로부터 얻는 단계의 총 갯수 (샘플의 batch) , np.ceil로 '올림'함


/tmp/ipykernel_155366/412469799.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))


For categoral classication the prediction will come with probability of each category. So we will pick the category that have the highest probability with numpy average max

In [106]:
predict

array([[1.0593922e-03, 9.9894065e-01],
       [9.8210984e-01, 1.7890163e-02],
       [9.4376391e-01, 5.6236096e-02],
       ...,
       [7.4775286e-02, 9.2522472e-01],
       [8.6558145e-01, 1.3441847e-01],
       [9.6814903e-05, 9.9990320e-01]], dtype=float32)

In [107]:
np.argmax(predict, axis=-1)

array([1, 0, 0, ..., 1, 0, 1])

In [108]:
test_df['category'] = np.argmax(predict, axis=-1) # axis에 해당하는 값들 중 가장 큰 값의 인덱스들을 반환하는 함수

In [109]:
test_df['category']

0        1
1        0
2        0
3        1
4        1
        ..
12495    1
12496    1
12497    1
12498    0
12499    1
Name: category, Length: 12500, dtype: int64

We will convert the predict category back into our generator classes by using `train_generator.class_indices`. It is the classes that image generator map while converting data into computer vision

In [110]:
dict((v,k) for k,v in train_generator.class_indices.items())

{0: 'cat', 1: 'dog'}

In [111]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map) # test_df['category'] 값을 0 or 1을 cat or dog로 바꿈

From our prepare data part. We map data with `{1: 'dog', 0: 'cat'}`. Now we will map the result back to dog is 1 and cat is 0

In [112]:
test_df

,filename,category
0,2196.jpg,dog
1,4285.jpg,cat
2,4373.jpg,cat
3,10108.jpg,dog
4,7030.jpg,dog
...,...,...
12495,3522.jpg,dog
12496,7509.jpg,dog
12497,6361.jpg,dog
12498,701.jpg,cat


In [113]:
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

In [114]:
test_df

,filename,category
0,2196.jpg,1
1,4285.jpg,0
2,4373.jpg,0
3,10108.jpg,1
4,7030.jpg,1
...,...,...
12495,3522.jpg,1
12496,7509.jpg,1
12497,6361.jpg,1
12498,701.jpg,0
